In [ ]:
!unzip STAD_DATA.zip

Archive:  STAD_DATA.zip
   creating: STAD_MSI_TEST/
   creating: STAD_MSI_TRAIN/
   creating: STAD_MSS_TEST/
   creating: STAD_MSS_TRAIN/


## DATA DOWNLOAD LINK

In [29]:
from urllib import request 
import os
import zipfile
data_name = ['CRC_DX_TRAIN_MSS', 'CRC_DX_TRAIN_MSI', 'CRC_DX_TEST_MSS', 'CRC_DX_TEST_MSI'] 
front_link = 'https://zenodo.org/record/2530835/files/' 
back_link = '.zip?download=1' 


for idx, data_type in enumerate(data_name):
  # 코랩 아니면 아래 경로 수정
  if idx <= 1:
    os.chdir('/content/TCGA_DATA/CRC_TRAIN')
  else:
    os.chdir('/content/TCGA_DATA/CRC_TEST')
  
  link = front_link + data_type + back_link
  request.urlretrieve(link,data_type) 
  file_name = data_type + '.zip'
  zipfile.ZipFile(file_name).extractall() 
  os.remove(file_name)



KeyboardInterrupt: ignored

## DATA PATH 가져오기 

In [ ]:
from glob import glob
DATA_PATH_TRAIN_LIST = glob('./TCGA_DATA/CRC_TRAIN/*/*.png') 
DATA_PATH_TEST_LIST = glob('./TCGA_DATA/CRC_TEST/*/*.png')

## Custom DataSet 

In [ ]:
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform

class STAD_DataSet(Dataset):
    #data_path_list - 이미지 path 전체 리스트
    #label - 이미지 ground truth
    def __init__(self, data_path_list, classes, transform=None):
        self.path_list = data_path_list
        self.label = get_label(data_path_list)
        self.transform = transform
        self.classes = classes
        
    def get_label(data_path_list):
        label_list = []
        for path in data_path_list:
            # 뒤에서 두번째가 class다.
            label_list.append(path.split('/')[-2])
        return label_list

    def __len__(self):
        return len(self.path_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image = io.imread(self.path_list[idx])
        if self.transform is not None:
            image = self.transform(image)
        return image, self.classes.index(self.label[idx])